In [31]:
import numpy as np
from scipy.optimize import linprog

def generate_bids(n, m, p_bar):
    """
    Generate a sequence of random bids.
    :param n: Total number of bids.
    :param m: Number of items.
    :param p_bar: Ground truth price vector.
    :return: Array of bids.
    """
    bids = []
    for _ in range(n):
        a_k = np.random.choice([0, 1], size=m)  # Generate a_k
        pi_k = np.dot(p_bar, a_k) + np.random.normal(0, np.sqrt(0.2))  # Calculate bid price
        bids.append((a_k, pi_k))
    return bids

def solve_offline_lp(bids, m, b_i):
    c = -np.array([pi_k for _, pi_k in bids])  # Negative for maximization
    A = np.array([a_k for a_k, _ in bids]).T  # Transpose to match dimensions
    b = b_i * np.ones(m)

    # Solving the LP
    result = linprog(c, A_ub=A, b_ub=b, bounds=(0, 1), method='highs')

    if result.success:
        return -result.fun  # Revenue (negate because of maximization)
    else:
        raise ValueError("Offline LP did not converge")

def solve_partial_lp_dual(bids, k, n, b_i):
    # Objective function: maximize sum(pi_j * x_j) for j=1 to k
    c = -np.array([pi_k for _, pi_k in bids[:k]])

    # Constraints: sum(a_ij * x_j) <= (k/n) * b_i for all i
    A = np.array([a_k for a_k, _ in bids[:k]]).T
    b = (k / n) * np.array(b_i)

    # Bounds for decision variables: 0 <= x_j <= 1
    x_bounds = [(0, 1) for _ in range(k)]

    # Solve the linear program
    result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

    # print(result.ineqlin.get('marginals'))

    if result.success:
        # The dual variable corresponding to the inequality constraints A_ub * x <= b_ub
        return result.get('slack'), -1*result.ineqlin.get('marginals'), result.get('fun')
    else:
        raise ValueError("failed to find a solution")

In [32]:
perf_dynamic_slpm = []
perf_ahdla = []
revenues = {}

n = 10000  # Total number of bids
m = 10     # Number of items
b_i = np.ones(m) * 1000  # Bid cap for all i
# Fixed ground truth price vector (p_bar) - set to ones for simplicity
p_bar = np.ones(m)  # Vector of ones
k_values = [50, 100, 200]  # Different k values to test
# Regenerate bids with the fixed p_bar
bids_fixed = generate_bids(n, m, p_bar)

# offline
offline_revenue_value = solve_offline_lp(bids_fixed, m, b_i)
print(f"Offline revenue: {offline_revenue_value}")

Offline revenue: 11312.492401325151


In [33]:
# Problem 1
def run_slpm_static(bids, k, n, b_i):
    revenue = 0
    remaining_capacity = np.array(b_i)
    # Solve the partial LP for the first k bids to get dual prices
    slack, dual_price, k_revenue = solve_partial_lp_dual(bids, k, n, b_i)
    # revenue += -k_revenue
    # remaining_capacity = np.array(b_i-((k/n)*b_i-slack))

    for i, (a_k, pi_k) in enumerate(bids):
        if i >= k:
            # Allocate based on the decision rule using y_bar
            if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
                revenue += pi_k
                remaining_capacity -= a_k

    return revenue

for k in k_values:
    revenue = run_slpm_static(bids_fixed, k, n, b_i)
    revenues[k] = revenue

for slpm_revenue, k in zip(revenues.values(), k_values):
    print(f"SLPM Static revenue: {slpm_revenue} at k={k}")

SLPM Static revenue: 9423.49276434075 at k=50
SLPM Static revenue: 8983.917563812547 at k=100
SLPM Static revenue: 10051.354283449951 at k=200


The increase in revenue as k grows suggests that having more bidder information upfront (i.e., a larger k) allows the algorithm to make more informed decisions which result in higher revenue.

It may have two tradeoffs here, one is tradeoff between cost of computation and accuracy, since with a large k we will need more compuration resources. Another tradeoff is that a larger k implies a delay in decision-making since more bids must be collected before making allocations, which could be a trade-off in real-time scenarios.


In [34]:
# Problem 2
def run_slpm_dynamic(bids, k, n, b_i, offline_revenue_value):
    global diff_dplm
    revenue = 0
    remaining_capacity = np.array(b_i)

    # Solve the partial LP for the first k bids to get dual prices
    slack, dual_price, k_revenue = solve_partial_lp_dual(bids, k, n, b_i)
    print(f"i={k}\ndual price: {dual_price}")

    target = 2*k

    for i, (a_k, pi_k) in enumerate(bids):
        if i >= k:
            if i == target:
                slack, dual_price, _ = solve_partial_lp_dual(bids, target, n, b_i)
                print(f"i={i}\ndual price: {dual_price}")
                target *= 2
            # Allocate based on the decision rule using y_bar
            if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
                revenue += pi_k
                remaining_capacity -= a_k
            perf_dynamic_slpm.append(revenue-((i+1)/n)*offline_revenue_value)


    return revenue

revenue = run_slpm_dynamic(bids_fixed, 50, n, b_i, offline_revenue_value)
print(f"SLPM Dynamic revenue: {revenue}")

i=50
dual price: [0.88491677 1.09396077 1.41107317 1.32719615 0.9998923  1.19161588
 1.057338   0.85257739 0.48138874 0.89137056]
i=100
dual price: [0.7639446  1.12288168 1.13240578 1.06072602 1.24343778 0.75718131
 1.21093843 1.02936733 0.85190305 1.29291199]
i=200
dual price: [1.00479909 1.03334461 1.11366267 1.04129026 1.11528736 0.9246298
 0.97376202 1.21458549 0.95116067 1.27235704]
i=400
dual price: [1.02126168 1.0046499  1.1032581  1.03170512 1.05198258 0.93210437
 1.00789511 1.18256298 1.09956477 1.1491772 ]
i=800
dual price: [1.10766248 1.04555501 1.14955738 1.02551813 1.06015404 1.05537855
 1.01811383 1.05525708 1.04921059 1.06302947]
i=1600
dual price: [1.04388764 1.07582885 1.16556891 1.08540075 1.04795455 1.05328224
 1.00964954 1.05144504 1.05335659 1.09404349]
i=3200
dual price: [1.05000085 1.06243157 1.10253013 1.09060167 1.05702651 1.05846012
 1.04647594 1.06018906 1.05125301 1.0981574 ]
i=6400
dual price: [1.04171665 1.06390252 1.10850432 1.06903605 1.08232444 1.067256

In this question, instead of using a fixed dual price, we update our dual price on k= [50, 100, 200, 400, 800, ...], and using this stratege, we can get better performance than with fixed dual price, 10905.843288804756 compare to revenue in question above.

Also we print all dual price on each update time point. As we can see, it approches to the ground true price p = [1,1,...,1,1] we set before.

In [35]:
# Problem 3
def run_ahdla(bids, k, n, b_i, offline_revenue_value):
    global diff_ahdla
    batch = 50
    revenue = 0
    dual_price = np.zeros(m)
    remaining_capacity = b_i
    target = batch*k

    for i, (a_k, pi_k) in enumerate(bids):
        # Allocate based on the decision rule using y_bar
        if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
            revenue += pi_k
            remaining_capacity -= a_k
        perf_ahdla.append(revenue-((i+1)/n)*offline_revenue_value)
        if i == target and i < n-1:
            _, dual_price, _ = solve_partial_lp_dual(bids, i+1, n, (n/(n-i-1))*remaining_capacity)
            if i % 1000 == 0:
                print(f"i={i}\ndual price: {dual_price}")
            target += batch

    return revenue

revenue = run_ahdla(bids_fixed, 1, n, b_i, offline_revenue_value)
print(f"Action-history-dependent Learning Algorithm revenue: {revenue}")

i=1000
dual price: [1.09157935 1.08592957 1.14891369 1.08582954 1.02110046 1.04722526
 1.02637201 1.04896519 1.06192431 1.05896325]
i=2000
dual price: [1.04248313 1.07167883 1.14851722 1.08768912 1.03299975 1.05705393
 1.04620513 1.05832449 1.0340698  1.11287801]
i=3000
dual price: [1.05205964 1.07175377 1.09502561 1.08279849 1.04880256 1.07931298
 1.07103537 1.06417773 1.05335659 1.10047727]
i=4000
dual price: [1.04438653 1.06535137 1.08928452 1.09144774 1.06965476 1.06384572
 1.07875102 1.06268189 1.04195222 1.09178549]
i=5000
dual price: [1.03211084 1.0779157  1.06211188 1.08539227 1.09007181 1.09728201
 1.08258659 1.05879774 1.04692652 1.0739885 ]
i=6000
dual price: [1.04325671 1.04578243 1.07825323 1.053795   1.10953241 1.1276712
 1.08885862 1.05297108 1.07443228 1.06461251]
i=7000
dual price: [1.01935069 1.06423617 1.0898612  1.10058056 1.09556867 1.09284747
 1.08396007 1.04746195 1.07523748 1.06651828]
i=8000
dual price: [1.03761885 1.08237367 1.04645869 1.07842057 1.11168905 1.

In [36]:
print(f"Performance of Dynamic SLPM Algorithm: {perf_dynamic_slpm[-20:]}") # print first 20 results
print(f"Performance of Action-history-dependent Learning Algorithm: {perf_ahdla[-20:]}") # print first 20 results

Performance of Dynamic SLPM Algorithm: [-294.12195461732153, -295.2532038574536, -296.3844530975857, -297.5157023377178, -298.6469515778517, -299.7782008179838, -300.9094500581159, -302.040699298248, -303.1719485383819, -304.303197778514, -305.4344470186461, -306.5656962587782, -307.6969454989103, -308.8281947390424, -309.9594439791763, -311.0906932193084, -312.2219424594423, -313.3531916995744, -314.4844409397065, -315.61569017983857]
Performance of Action-history-dependent Learning Algorithm: [-41.493671702366555, -42.62492094249865, -43.75617018263074, -44.88741942276283, -46.018668662896744, -47.149917903028836, -48.28116714316093, -49.41241638329302, -50.54366562342693, -51.674914863559025, -52.80616410369112, -53.93741334382321, -55.0686625839553, -56.199911824087394, -57.331161064221305, -58.4624103043534, -59.59365954448731, -60.7249087846194, -61.856158024751494, -62.987407264883586]


In Action-history-dependent Learning Algorithm, instead of updating dual price in a frequency of [50, 100, 200, ...], we update it for each batch, we set 50 here. And as we can see, the revenue is higher than what we got from problem 2.

Hence Action-history-dependent Learning Algorithm have better performance. Becuase it also update with remaining capacity information.

As we can see in the difference vector, compared with problem 2, (see result on problem2), Action-history-dependent Learning Algorithm is closer to partial of optimal solution at each time point.

# Problem 4
## Convexity

The Formulation (3) in question is as follows:

\begin{align*}
\text{minimize}_{\bar{\mathbf{y}}} \; & \;\mathbf{d}^T \bar{\mathbf{y}} + \mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+ \\
\text{s.t. } & \bar{\mathbf{y}} \geq 0
\end{align*}

where $\mathbf{d} = \frac{\mathbf{b}}{n}$ and $(\cdot)^+ = \max \{\cdot, 0\}$.

To identify whether (3) is a convex optimization problem or not, we can analyze each part of (3) separately.

For the first part $\mathbf{d}^T \bar{\mathbf{y}}$ of (3), $\mathbf{d}^T \bar{\mathbf{y}}$ is liner in $\bar{\mathbf{y}}$. We know that linear functions are both convex and concave.

For the second part $\mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+$, since $\pi - \mathbf{a}^T \bar{\mathbf{y}}$ is linear in $\bar{\mathbf{y}}$. What $(\cdot)^+$ does is to get maximum value between this linear function and zero, since both of them are convex, $\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+$ is also convex. The last part we need to consider is the expectation. From Jensen's inequality:

$$\mathbb{E}[f(X)]≥f(\mathbb{E}[X])$$

we know that if $f(x)$ is convex, then $\mathbb{E}[f(x)]$ is also convex. Therefore, $\mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+$ is convex.

Since both parts of (3) are convex, we know that (3) is a convex optimization problem.

## Connection

The dual problem of (1) is:

$$
\min \sum_{i=1}^{m} b_iy_i + \sum_{j=1}^{n} \beta_j\\
\text{s.t.} \sum_{i=1}^{m} a_{ij}y_i + \beta_j \geq \pi_j, \quad j=1,\ldots,n.\\
y_i, \beta_j \geq 0 \text{ for all } i, j
$$

From previous questions, we know that the primal optimal solution satisfies:
$$
x_j^* =
\begin{cases}
1, & \text{if } \pi_j > a_j^T y_k^* \\
0, & \text{if } \pi_j \leq a_j^T y_k^*.
\end{cases}
$$

the optimal solution $x_j$ may take non-integer values. That means the optimal solution of primary problem (1) highly depends on $y$. So by plugging the constraints $\sum_{i=1}^{m} a_{ij}y_i + \beta_j \geq \pi_j$ into the objective function, an equivalent form of the dual problem can be obtained as:
$$
\min \sum_{i=1}^{m} b_i y_i + \sum_{j=1}^{n} \left( \pi_j - \sum_{i=1}^{m} a_{ij} y_i \right)^+\\
\text{s.t. } y_i \geq 0, \quad i = 1, \ldots, m.
$$

And if we devide it by $n$, from above we know that $\mathbf{d} = \frac{\mathbf{b}}{n}$, so we now have:
$$
\min f_n(\bar{\mathbf{y}}) = \sum_{i=1}^{m} d_i y_i + \frac{1}{n} \sum_{j=1}^{n} \left( \pi_j - \sum_{i=1}^{m} a_{ij} y_i \right)^+\\
\text{s.t. } y_i \geq 0, \quad i = 1, \ldots, m.
$$

Since $(\pi, a)$ is a sequence of i.i.d. random vectors, we can also write this formulation as:

$$
\begin{align*}
\text{minimize}_{\bar{\mathbf{y}}} \; & \;\mathbf{d}^T \bar{\mathbf{y}} + \mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+ \\
\text{s.t. } & \bar{\mathbf{y}} \geq 0
\end{align*}
$$

where the expectation is taken with respect to $(\pi, a)$.
